# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 5: Algoritmo genético
Indicaciones previas:
- Las respuestas deben tener un buen fundamento teórico, se realizarán descuentos en el puntaje a respuestas que no contesten a lo solicitado
- Cualquier indicio de plagio resultará en la anulación de la prueba.

La tarea de este laboratorio consiste en comparar métodos en la elaboración de un algoritmo genético para la resolución de la asignación de vuelos.<br>Al final de este notebook se encuentran las preguntas que serán evaluadas en este laboratorio. 


#### Representacion de Individuo:

Un objeto Individual representaa una determinada asignacion de un subconjunto de vuelos (almacenado en <b>list_of_flights</b>) a un conjunto de gates (almacenado en <b>chromosome</b>). La lista de gates disponibles que se pueden asignar a los vuelos se almacena en la variable <b>allele_pool</b>  del individuo. Ver el siguiente grafico ilustrativo:

<img src="EjemploIndividuo.png">

**Usted deberá completar el código en las secciones indicadas con ######### COMPLETAR #########**

In [1]:
from random import seed, randint, sample, uniform, randrange
from copy import deepcopy

### Clase <b>Individual</b>

Esta es una clase para definir a un individuo de la población. Cada individuo posee un cromosoma, los vuelos a asignar, todos los posibles alelos y su respectivo fitness. Además, los métodos de esta clase permiten realizar el cruzamiento (crossover) y la mutación (mutation) sobre el cromosoma del individuo.

In [2]:
class Individual(object):
  """ Clase que implementa el individuo y sus operadores. El cromosoma de un individuo es una lista de caracteres,
      cada elemento de la lista es un gen cuyos alelos (caracteres) posibles se indican en allele_pool"""

  def __init__(self, chromosome, list_of_flights, allele_pool):  # El constructor recibe el cromosoma y el pool de alelos posibles
    self.chromosome = chromosome[:]
    self.list_of_flights = list_of_flights[:]
    self.allele_pool = allele_pool
    self.fitness = -1  # -1 indica que el individuo no ha sido evaluado

  def crossover_onepoint(self, other):
    "Retorna dos nuevos individuos del cruzamiento de un punto entre individuos self y other "
    c = randrange(len(self.chromosome))
    ind1 = Individual(self.chromosome[:c] + other.chromosome[c:], self.list_of_flights, self.allele_pool)
    ind2 = Individual(other.chromosome[:c] + self.chromosome[c:], self.list_of_flights, self.allele_pool)
    return [ind1, ind2]
  
  def crossover_permutation(self, other):
    new_list = list(set(self.chromosome + other.chromosome))
    flights_to_assign = len(self.list_of_flights)
    ind1 = Individual(sample(new_list, flights_to_assign), self.list_of_flights, self.allele_pool)
    ind2 = Individual(sample(new_list, flights_to_assign), self.list_of_flights, self.allele_pool)
    return [ind1, ind2]
  
  def crossover_uniform(self, other):
    chromosome1 = []
    chromosome2 = []
    "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
    for i in range(len(self.chromosome)):
      if uniform(0, 1) < 0.5:
        chromosome1.append(self.chromosome[i])
        chromosome2.append(other.chromosome[i])
      else:
        chromosome1.append(other.chromosome[i])
        chromosome2.append(self.chromosome[i])
    ind1 = Individual(chromosome1, self.list_of_flights, self.allele_pool)
    ind2 = Individual(chromosome2, self.list_of_flights, self.allele_pool)
    return [ind1, ind2]

  def mutate_swap(self):
    "Escoge dos genes e intercambia sus alelos"
    mutated_chromosome = deepcopy(self.chromosome)
    mutGen1 = randrange(0, len(mutated_chromosome))
    mutGen2 = randrange(0, len(mutated_chromosome))
    temp = mutated_chromosome[mutGen1]
    mutated_chromosome[mutGen1] = mutated_chromosome[mutGen2]
    mutated_chromosome[mutGen2] = temp
    return Individual(mutated_chromosome, self.list_of_flights, self.allele_pool)
  
  def mutate_position(self):
    "Cambia aleatoriamente el alelo de un gen."
    mutated_chromosome = deepcopy(self.chromosome)
    ######### COMPLETAR #########
    mutGen1 = randrange(0, len(self.allele_pool))
    mutGen2 = randrange(0, len(mutated_chromosome))
    temp = self.allele_pool[mutGen1]
    mutated_chromosome[mutGen2] = temp
    
    return Individual(mutated_chromosome, self.list_of_flights, self.allele_pool)

### Clase <b>Gate</b>

Esta es una clase abstracta para definir los lugares donde los vuelos serán asignados. Se debe hacer subclases con el fin de diferenciar los tipos de lugar donde un vuelo puede ser asignado.

In [3]:
class Gate(object):
  def __init__(self, identifier, x, y, z):
    self.identifier = identifier
    self.distance = x
    self.potential_of_speed = y
    self.number_of_persons_every_10m = z
  
  def __hash__(self):
    return self.identifier

### Clase <b>Sleeve</b>

Esta clase implementa concretamente el Gate tipo Sleeve (Manga). En este tipo de gate, los pasajeros deben realizar un recorrido a pie hasta abandonar totalmente el Gate. Se tiene en cuenta la longitud de la manga, la velocidad de los pasajeros y la cantidad de personas que pueden estar cada 10m (este valor es variable porque ya refleja el ancho de la manga, puesto que una manga más ancha permite una mayor cantidad de pasajeros).

In [4]:
class Sleeve(Gate):
  def __init__(self, identifier, length_of_sleeve, speed_of_passengers_on_sleeve, number_of_passengers_every_10m):
    super().__init__(identifier, length_of_sleeve, speed_of_passengers_on_sleeve, number_of_passengers_every_10m)

### Clase <b>Zone</b>

Esta clase implementa concretamente el Gate tipo Zone (zona). En este tipo de gate, los pasajeros son recogidos por un bus y son llevados hasta una puerta. Se tiene en cuenta la distancia de la zona a la puerta, la velocidad y capacidad del bus.

In [5]:
class Zone(Gate):
  def __init__(self, identifier, distance_zone_door, speed_bus, capacity_of_bus):
    super().__init__(identifier, distance_zone_door, speed_bus, capacity_of_bus)

### Clase <b>Flight</b>

Esta es una clase para definir a los vuelos a asignar. Cada vuelo posee una capacidad máxima, la cantidad de pasajeros, el tiempo de estacionamiento que le toma al vuelo, la longitud de las alas del avión, el tiempo de inspección al avión luego de aterrizar, el tiempo que demorarían en bajar las escaleras los pasajeros si el avión estuviera repleto, el momento de llegada al aeropuerto y el momento en que debería irse del aeropuerto.

In [6]:
class Flight(object):
  def __init__(self, identifier, maximum_capacity, number_of_passengers, parking_time, length_wings, inspection_time, landing_time_on_stairs, arriving_time, leaving_time):
    self.identifier = identifier
    self.maximum_capacity = maximum_capacity
    self.number_of_passengers = number_of_passengers
    self.parking_time = parking_time
    self.length_wings = length_wings
    self.inspection_time = inspection_time
    self.landing_time_on_stairs = landing_time_on_stairs
    self.arriving_time = arriving_time
    self.leaving_time = leaving_time
  
  def __hash__(self):
    return self.identifier

### Funciones utilitarias para generar los Gates y Vuelos</b>
Estas son funciones utilitarias para generar automáticamente los Gates y Vuelos

In [7]:
def generate_list_of_gates(number_of_gates, max_length_of_sleeve=20, max_speed_of_passengers_on_sleeve=3, max_number_of_passengers_every_10m=12, max_distance_zone_door=50, max_speed_bus=15, max_capacity_of_bus=200):
  list_of_gates = list()

  for gate_identifier in range(number_of_gates):
    gate_type = randint(0, 1)

    if gate_type == 0:
      distance = randint(1, max_length_of_sleeve)
      potential_of_speed = randint(1, max_speed_of_passengers_on_sleeve)
      number_of_persons_every_10m = randint(1, max_number_of_passengers_every_10m)

      gate = Sleeve(gate_identifier, distance, potential_of_speed, number_of_persons_every_10m)
    else:
      distance = randint(1, max_distance_zone_door)
      potential_of_speed = randint(1, max_speed_bus)
      number_of_persons_every_10m = randint(1, max_capacity_of_bus)

      gate = Zone(gate_identifier, distance, potential_of_speed, number_of_persons_every_10m)
    
    list_of_gates.append(gate)
  
  return list_of_gates

def generate_list_of_flights(number_of_flights, max_maximum_capacity=100, max_number_of_passengers_factor=0.8, max_parking_time=30, max_length_wings=25, max_inspection_time=180, max_landing_time_on_stairs=60, max_arriving_time=200, max_leaving_time=1000):
  list_of_flights = list()

  max_number_of_passengers = max_number_of_passengers_factor * max_maximum_capacity

  for flight_identifier in range(number_of_flights):
    maximum_capacity = randint(max_number_of_passengers, max_maximum_capacity)
    number_of_passengers = randint(1, max_number_of_passengers)
    parking_time = randint(1, max_parking_time)
    length_wings = randint(1, max_length_wings)
    inspection_time = randint(1, max_inspection_time)
    landing_time_on_stairs = randint(1, max_landing_time_on_stairs)
    arriving_time = randint(1, max_arriving_time)
    leaving_time = randint(max_arriving_time + 1, max_leaving_time)

    flight = Flight(flight_identifier, maximum_capacity, number_of_passengers, parking_time, length_wings, inspection_time, landing_time_on_stairs, arriving_time, leaving_time)
    
    list_of_flights.append(flight)
  
  return list_of_flights

### Funciones utilitarias para ordenar los Vuelos y Gates</b>
Estas son funciones utilitarias para ordenar los Vuelos acorde a su deseabilidad y los Gates por su flujo personas

In [8]:
def process_desirability(n):
  return n.number_of_passengers / (n.leaving_time - n.arriving_time)

def process_flow(n):
  return n.number_of_persons_every_10m * n.potential_of_speed * 10 / n.distance

### Funciones utilitarias para el Algoritmo Genético</b>
Estas son funciones utilitarias para realizar el Algoritmo Genético que se encargue de la asignación de vuelos

In [9]:
# Inicialización aleatoria de la población
def init_population(population_number, flights_to_assign, allele_pool, list_of_flights):
  population = []
  for i in range(population_number):
    new_chromosome_for_individual = sample(allele_pool, flights_to_assign)
    list_of_flights_for_individual = sample(list_of_flights, flights_to_assign)
    population.append( Individual(new_chromosome_for_individual, list_of_flights_for_individual, allele_pool) )
  return population

# Funcion de fitness: evalua un individuo dada su lista de vuelos (solution_flights) y los gates asignados (solution_gates)
# El fitness representa la suma total de ratios de numeros de pasajeros desembarcados por unidad de tiempo en cada gate asignado  
def fitness(solution_flights, solution_gates):
  grace_time = 5/100

  cumulative_time = 0
  for flight, gate in zip(solution_flights, solution_gates):
    number_of_passengers = flight.number_of_passengers
    if isinstance(gate, Sleeve):
      disembarkation_time_of_passengers = ( (number_of_passengers * (gate.distance + flight.length_wings) / process_flow(gate)) + flight.parking_time + flight.inspection_time ) * (1 + grace_time)
    else:
      disembarkation_time_of_passengers = ( (number_of_passengers * gate.distance / process_flow(gate)) + flight.parking_time + (flight.landing_time_on_stairs * number_of_passengers / flight.maximum_capacity) + flight.inspection_time ) * (1 + grace_time)
    cumulative_time += number_of_passengers / disembarkation_time_of_passengers
  return cumulative_time

# Evaluar la población con la función fitness
def evaluate_population(population, fitness_fn):
  """ Evalua una poblacion de individuos con la funcion de fitness pasada """
  population_size = len(population)
  for i in range(population_size):
    if population[i].fitness == -1:    # Evalúa sólo si el individuo no esta evaluado
      population[i].fitness = fitness_fn(population[i].list_of_flights, population[i].chromosome)

# Selección de parientes por el método roulette
def select_parents_roulette(population):
  population_size = len(population)
  
  sumfitness = sum([indiv.fitness for indiv in population])
  pickfitness = uniform(0, sumfitness)
  cumfitness = 0
  for i in range(population_size):
    cumfitness += population[i].fitness
    if cumfitness > pickfitness: 
      iParent1 = i
      break
  
  sumfitness = sumfitness - population[iParent1].fitness
  pickfitness = uniform(0, sumfitness)
  cumfitness = 0
  for i in range(population_size):
    if i == iParent1:
      continue
    cumfitness += population[i].fitness
    if cumfitness > pickfitness: 
      iParent2 = i
      break
  return (population[iParent1], population[iParent2])

# Selección de parientes por el método tournament
def select_parents_tournament(population, percentage_size):
  population_size = len(population)
  tournament_size = int(percentage_size*population_size/100)
  

  # Escoge el primer padre
  tournament_pool = sample(population, tournament_size)
  max_fitness = -1
  for i in range(len(tournament_pool)):  # recorre los individuos que participan en el torneo buscando el mejor individuo
    if tournament_pool[i].fitness > max_fitness:
      max_fitness = tournament_pool[i].fitness
      iParent1 = i
  
  # Escoge el segundo padre
  ######### COMPLETAR #########
  tournament_pool = sample(population, tournament_size)
  max_fitness = -1
  for i in range(len(tournament_pool)):  # recorre los individuos que participan en el torneo buscando el mejor individuo
    if tournament_pool[i].fitness > max_fitness and i != iParent1:
      max_fitness = tournament_pool[i].fitness
      iParent2 = i
  return (population[iParent1], population[iParent2])

# Selección de la nueva población
def select_survivors(population, offspring_population, numsurvivors):
  next_population = []
  population.extend(offspring_population) # Une las dos poblaciones
  survivors = sorted(population, key=lambda x: x.fitness, reverse=True)[:numsurvivors]
  next_population.extend(survivors)
  return next_population

def genetic_algorithm(list_of_flights, list_of_gates, num_individuals, flights_to_assign, fitness_fn, n_generations, selection_fn="roulette", crossover="onepoint", mutation="position", percentage_size=5, p_mut=0.05):
  seed(0)

  allele_pool = list_of_gates

  #Inicializa una poblacion inicial de forma aleatoria
  population = init_population(num_individuals, flights_to_assign, allele_pool, list_of_flights)

  population_size = len(population)
  ######### COMPLETAR #########
  evaluate_population(population,fitness_fn)
  
  for _ in range(n_generations):  # Por cada generacion
     ## Selecciona las parejas de padres para cruzamiento
    mating_pool = []
    for i in range(int(population_size/2)):
      if selection_fn == "roulette":
        ######### COMPLETAR #########
        mating_pool.append(select_parents_roulette(population))
      elif selection_fn == "tournament":
        mating_pool.append(select_parents_tournament(population, percentage_size))

    ## Crea la poblacion descendencia cruzando las parejas del mating pool    
    offspring_population = []
    for i in range(len(mating_pool)):
      if crossover == "onepoint":
        offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) )
      elif crossover == "permutation":
        ######### COMPLETAR #########
        offspring_population.extend( mating_pool[i][0].crossover_permutation(mating_pool[i][1]) )
      elif crossover == "uniform":
        ######### COMPLETAR #########
        offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) )
    
    ## Aplica el operador de mutacion con probabilidad p_mut en cada hijo generado
    for i in range(len(offspring_population)):
      if uniform(0, 1) < p_mut: 
        if mutation == "swap":
          ######### COMPLETAR #########
          offspring_population[i] = offspring_population[i].mutate_swap() 
        elif mutation == "position":
          offspring_population[i] = offspring_population[i].mutate_position()

    ######### COMPLETAR #########
    evaluate_population(offspring_population,fitness_fn)
    population=select_survivors(population,offspring_population,population_size)
  
  best = sorted(population, key=lambda x: x.fitness, reverse=True)[0]

  return best

### Funciones utilitarias para manejar la solución del Algoritmo Genético</b>
Esta es una función utilitarias que permite mostrar la configuración usada en el Algoritmo Genético y la asignación de Vuelos a Gates encontrada en la solución

In [10]:
def make_solution_report(solution, selection_fn, crossover, mutation):
  print("Selection: {} - Crossover: {} - Mutation: {} - Fitness: {}".format(selection_fn, crossover, mutation, solution.fitness), flush=True)
  print(" - ".join(["Flight {} in {} {}".format(flight.identifier, "Zone" if isinstance(gate, Zone) == True else "Sleeve", gate.identifier) for flight, gate in zip(solution.list_of_flights, solution.chromosome)]) + "\n", flush=True)

## <b>Algoritmo Genético</b> 

### <b>Generación de Gates y Vuelos</b> 

Implementación para generar los Gates y Vuelos, **sin filtrar los primeros N vuelos más deseables que pueden ser asignados, dejando el trabajo de selección de vuelos al Algoritmo Genético**

In [11]:
seed(0)

number_of_gates = 7
number_of_flights = 20

list_of_gates = generate_list_of_gates(number_of_gates)
list_of_flights = generate_list_of_flights(number_of_flights)

number_of_gates = len(list_of_gates)
number_of_flights = len(list_of_flights)

flights_to_assign = number_of_flights if number_of_flights <= number_of_gates	else number_of_gates
  
list_of_flights = sorted(list_of_flights, key=process_desirability, reverse=True)
list_of_gates = sorted(list_of_gates, key=process_flow, reverse=True)

### <b>Experimentación con el Algoritmo Genético</b> 

In [12]:
num_individuals = 100

fitness_fn = fitness
n_generations = 20

best_fitness = 0

selection_fn = "roulette"
crossover = "onepoint"
mutation = "position"

solution = genetic_algorithm(list_of_flights, list_of_gates, num_individuals, flights_to_assign, fitness_fn, n_generations, selection_fn, crossover, mutation)

make_solution_report(solution, selection_fn, crossover, mutation)

Selection: roulette - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4



In [13]:
num_individuals = 100

fitness_fn = fitness
n_generations = 20

best_fitness = 0

######### COMPLETAR #########
for selection_fn in ["roulette", "tournament"]:  # por cada operador de seleccion
  for crossover in ["onepoint","uniform","permutation"]:    # por cada operador de cruzamiento
    for mutation in ["position","swap"]:   # por cada operador de mutacion
        solution=genetic_algorithm(list_of_flights, list_of_gates, num_individuals, flights_to_assign, fitness_fn, n_generations, selection_fn, crossover, mutation)
        make_solution_report(solution, selection_fn, crossover, mutation)
        if best_fitness<solution.fitness:
            best_fitness=solution.fitness

print("Best Fitness: {}".format(best_fitness))

Selection: roulette - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: onepoint - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: rou

# Preguntas:

**1.** Completar el código

**(4 Puntos)**

**2.** Para la "Generación de Gates y Vuelos", experimentar con el algoritmo genético usando las siguientes configuraciones:

- selection_fn: "roulette", "tournament"
- crossover: "onepoint", "uniform", "permutation"
- mutation: "position", "swap"

En total son 12 combinaciones a realizar. En cada configuración se debe usar la función "make_solution_report" para obtener los resultados. La lista de Vuelos y Gates sólo se generan 1 vez, a partir de las cuales se experimenta con cada configuración del Algoritmo Genético.

**No modificar los valores de "percentage_size" y "p_mut" al ejecutar el algoritmo genético**

**Se debe mostrar el código respectivo y la solución generada en cada configuración o se considerará como inválido. Para este fin, puede agregar celdas a su gusto.**

**(5 Puntos, cada 3 configuraciones realizadas equivale a 1 punto)**

**3.** Para la "Generación de Gates y Vuelos no restringida", seleccionar las soluciones con el mejor fitness.

**En total debería tener 5 configuraciones con el mejor fitness (contando todos los decimales devueltos en la impresión de make_solution_report**

**(5 Puntos, cada configuración explicada equivale a 1 punto)**

**4.** Por cada solución (de la pregunta 3), explicar teóricamente porque dicha configuración ("selection_fn", "crossover" y "mutation") obtiene el mejor fitness. Se debe tener en cuenta que en este caso el Algoritmo Genético se encargó de elegir los vuelos a ser asignados.

**Es necesario que indique en que parte del código el Algoritmo Genético eligió los vuelos, o las justificaciones teóricas no se considerarán válidas.**

**Se debe explicar teóricamente cada configuración por separado, o se considerará como inválido.**

**En total debería tener 5 configuraciones con el mejor fitness (contando todos los decimales devueltos en la impresión de make_solution_report**

**(6 Puntos, indicar la parte del código el Algoritmo Genético eligió los vuelos equivale 1 punto, y cada configuración explicada equivale a 1 punto)**

In [ ]:
Pregunta 2 codigo:

for selection_fn in ["roulette", "tournament"]:  # por cada operador de seleccion
  for crossover in ["onepoint","uniform","permutation"]:    # por cada operador de cruzamiento
    for mutation in ["position","swap"]:   # por cada operador de mutacion
        solution=genetic_algorithm(list_of_flights, list_of_gates, num_individuals, flights_to_assign, fitness_fn, n_generations, selection_fn, crossover, mutation)
        make_solution_report(solution, selection_fn, crossover, mutation)
        if best_fitness<solution.fitness:
            best_fitness=solution.fitness

print("Best Fitness: {}".format(best_fitness))

pREGUNTA 2 RESULTADOS:
    
Selection: roulette - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: onepoint - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: permutation - Mutation: position - Fitness: 3.4990975120183485
Flight 14 in Zone 2 - Flight 6 in Zone 2 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: permutation - Mutation: swap - Fitness: 3.4932820489079277
Flight 14 in Sleeve 4 - Flight 6 in Zone 2 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 0 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: onepoint - Mutation: swap - Fitness: 3.4614542164806985
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Sleeve 4 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: uniform - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: uniform - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: permutation - Mutation: position - Fitness: 2.9355760195147536
Flight 9 in Sleeve 4 - Flight 19 in Sleeve 6 - Flight 12 in Zone 0 - Flight 15 in Zone 0 - Flight 10 in Sleeve 5 - Flight 3 in Zone 2 - Flight 14 in Sleeve 1

Selection: tournament - Crossover: permutation - Mutation: swap - Fitness: 3.12375632333688
Flight 9 in Sleeve 4 - Flight 19 in Sleeve 4 - Flight 12 in Sleeve 4 - Flight 15 in Zone 2 - Flight 10 in Zone 2 - Flight 3 in Zone 2 - Flight 14 in Sleeve 4

Best Fitness: 3.500583564295903

Pregunta 3:

Selection: roulette - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: onepoint - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: roulette - Crossover: uniform - Mutation: swap - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Selection: tournament - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903
Flight 14 in Sleeve 4 - Flight 6 in Sleeve 4 - Flight 15 in Zone 2 - Flight 1 in Zone 2 - Flight 2 in Zone 2 - Flight 11 in Sleeve 4 - Flight 9 in Sleeve 4

Pregunta 4:
En primer lugar mencionaremos en que parte del codigo se eligen los vuelos.

Los vuelos se escogen en list_of_flights_for_individual = sample(list_of_flights, flights_to_assign) de init_population(population_number, flights_to_assign, allele_pool, list_of_flights). En esta parte se escoge una parte de los vuelos para posteriormente asignarlos a la poblacion inicial. Es importante mencionar que luego de terminar el algoritmo genetico quedaran las poblaciones 
sobrevivientes.

Configuracion 1:

Selection: roulette - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903

En roullete se les atribuye a los individuos pedazos de una ruleta, cuyo ancho es proporcional a su fitnes, esta ruleta gira n veces para seleccionar n progenitores

Posteriormente se realiza un crossover onepoint en donde se escoge un punto aleatorio de corte y a partir de ahi cambian se reparten los valores de los padres, siendo los de uno desde el inicio hasta el punto de corte y del otro desde el punto de corte hasta el final. Finalmente se realiza una mutacion por posicion, aqui cambiamos aleatoriamente el alelo de un gen.

Configuracion 2:

Selection: roulette - Crossover: onepoint - Mutation: swap - Fitness: 3.500583564295903

En roullete se les atribuye a los individuos pedazos de una ruleta, cuyo ancho es proporcional a su fitnes, esta ruleta gira n veces para seleccionar n progenitores

Posteriormente se realiza un crossover onepoint en donde se escoge un punto aleatorio de corte y a partir de ahi cambian se reparten los valores de los padres, siendo los de uno desde el inicio hasta el punto de corte y del otro desde el punto de corte hasta el final. Finalmente se realiza una mutacion por intercambio en donde escogemos dos genes e intercambia sus alelos.


Configuracion 3:
Selection: roulette - Crossover: uniform - Mutation: position - Fitness: 3.500583564295903

En roullete se les atribuye a los individuos pedazos de una ruleta, cuyo ancho es proporcional a su fitnes, esta ruleta gira n veces para seleccionar n progenitores. 

Posteriormente realizamos un crossover uniform en donde escogemos un gen de uno de los padres para un hijo y por consecuencia el gen del otro padre sera para el otro hijo en la misma posicion.
 Finalmente se realiza una mutacion por posicion, aqui cambiamos aleatoriamente el alelo de un gen.



Configuracion 4:

Selection: roulette - Crossover: uniform - Mutation: swap - Fitness: 3.500583564295903

En roullete se les atribuye a los individuos pedazos de una ruleta, cuyo ancho es proporcional a su fitnes, esta ruleta gira n veces para seleccionar n progenitores. 

Posteriormente realizamos un crossover uniform en donde escogemos un gen de uno de los padres para un hijo y por consecuencia el gen del otro padre sera para el otro hijo en la misma posicion.Finalmente se realiza una mutacion por intercambio en donde escogemos dos genes e intercambia sus alelos.


Configuracion 5:

Selection: tournament - Crossover: onepoint - Mutation: position - Fitness: 3.500583564295903

Primero realizamos una seleccion por torneo en donde se selecciona k individuos aleatoriamente y se eligen los que tienen mayor fitness enfrentandolos uno a uno. Esto se repite n veces.

Posteriormente se realiza un crossover onepoint en donde se escoge un punto aleatorio de corte y a partir de ahi cambian se reparten los valores de los padres, siendo los de uno desde el inicio hasta el punto de corte y del otro desde el punto de corte hasta el final. Finalmente se realiza una mutacion por posicion, aqui cambiamos aleatoriamente el alelo de un gen.


Observacion:

Es importante mencionar que todas estas configuraciones tienen el mismo fitness dado que nos estan devolviendo exactamente las mismas combinaciones en el cromosoma y justo se calcula el fitness de esa combinacion que es la optima para esa poblacion. Esto pasa porque los vuelos y los gates se van a asignar a en un conjunto finito.
